# Задача 1

Модифицируйте логирование, чтобы выводились не пошаговые значения потерь, а средние значения за эпоху.

In [ ]:
import os
if not os.path.exists('open-images-bus-trucks'):
    !pip install -q torch_snippets==0.5 # https://pypi.org/project/torch-snippets/#history здесь история версий библиотеки
    !wget --quiet https://www.dropbox.com/s/agmzwk95v96ihic/open-images-bus-trucks.tar.xz
    !tar -xf open-images-bus-trucks.tar.xz
    !rm open-images-bus-trucks.tar.xz
    !git clone https://github.com/sizhky/ssd-utils/
%cd ssd-utils

Cloning into 'ssd-utils'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (9/9), 13.65 KiB | 13.65 MiB/s, done.
/content/ssd-utils/ssd-utils/ssd-utils


In [ ]:
# Перейти в папку с кодом модели

# Импортировать модель SSD и функцию потерь
from model import SSD300, MultiBoxLoss
from detect import *


In [ ]:
DATA_ROOT = '/content/open-images-bus-trucks/'
IMAGE_ROOT = f'{DATA_ROOT}/images'
DF_RAW = pd.read_csv(f'{DATA_ROOT}/df.csv')

df = DF_RAW.copy()
df = df[df['ImageID'].isin(df['ImageID'].unique().tolist())]

label2target = {l: t+1 for t, l in enumerate(DF_RAW['LabelName'].unique())}
label2target['background'] = 0
target2label = {t: l for l, t in label2target.items()}
background_class = label2target['background']
num_classes = len(label2target)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

FileNotFoundError: [Errno 2] No such file or directory: '/content/open-images-bus-trucks//df.csv'

In [ ]:
n_epochs = 3
model = SSD300(num_classes, device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
criterion = MultiBoxLoss(priors_cxcy=model.priors_cxcy, device=device)

train_loss_history = []
val_loss_history = []

for epoch in range(n_epochs):
    model.train()
    train_losses_epoch = []
    for inputs in train_loader:
        loss = train_batch(inputs, model, criterion, optimizer)
        train_losses_epoch.append(loss.item())
    avg_train_loss = np.mean(train_losses_epoch)
    train_loss_history.append(avg_train_loss)

    model.eval()
    val_losses_epoch = []
    for inputs in test_loader:
        loss = validate_batch(inputs, model, criterion)
        val_losses_epoch.append(loss.item())
    avg_val_loss = np.mean(val_losses_epoch)
    val_loss_history.append(avg_val_loss)

    print(f"Epoch {epoch+1}/{n_epochs} - Train Loss: {avg_train_loss:.4f} - Val Loss: {avg_val_loss:.4f}")


NameError: name 'num_classes' is not defined

In [ ]:
from torch_snippets import Report

n_epochs = 3
model = SSD300(num_classes, device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
criterion = MultiBoxLoss(priors_cxcy=model.priors_cxcy, device=device)

log = Report(n_epochs=n_epochs)

for epoch in range(n_epochs):
    # === ОБУЧЕНИЕ ===
    train_losses_epoch = []
    for inputs in train_loader:
        loss = train_batch(inputs, model, criterion, optimizer)
        train_losses_epoch.append(loss.item())
    avg_train_loss = np.mean(train_losses_epoch)
    log.record(epoch + 1, trn_loss=avg_train_loss, end=' ')

    # === ВАЛИДАЦИЯ ===
    val_losses_epoch = []
    for inputs in test_loader:
        loss = validate_batch(inputs, model, criterion)
        val_losses_epoch.append(loss.item())
    avg_val_loss = np.mean(val_losses_epoch)
    log.record(epoch + 1, val_loss=avg_val_loss, end='\n')


ImportError: cannot import name 'Report' from 'torch_snippets' (/usr/local/lib/python3.11/dist-packages/torch_snippets/__init__.py)

In [ ]:
n_epochs = 3
model = SSD300(num_classes, device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
criterion = MultiBoxLoss(priors_cxcy=model.priors_cxcy, device=device)

train_loss_history = []
val_loss_history = []

for epoch in range(n_epochs):
    model.train()
    train_losses_epoch = []
    for inputs in train_loader:
        loss = train_batch(inputs, model, criterion, optimizer)
        train_losses_epoch.append(loss.item())
    avg_train_loss = np.mean(train_losses_epoch)
    train_loss_history.append(avg_train_loss)

    model.eval()
    val_losses_epoch = []
    for inputs in test_loader:
        loss = validate_batch(inputs, model, criterion)
        val_losses_epoch.append(loss.item())
    avg_val_loss = np.mean(val_losses_epoch)
    val_loss_history.append(avg_val_loss)

    print(f"Epoch {epoch+1}/{n_epochs} - Train Loss: {avg_train_loss:.4f} - Val Loss: {avg_val_loss:.4f}")


NameError: name 'SSD300' is not defined

In [ ]:
%cd /content/ssd-utils


[Errno 2] No such file or directory: '/content/ssd-utils'
/content


In [ ]:
from model import SSD300, MultiBoxLoss
n_epochs = 3
model = SSD300(num_classes, device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
criterion = MultiBoxLoss(priors_cxcy=model.priors_cxcy, device=device)

for epoch in range(n_epochs):
    model.train()
    epoch_losses = []

    for inputs in train_loader:
        images, boxes, labels = inputs
        images = images.to(device)
        boxes = [b.to(device) for b in boxes]
        labels = [l.to(device) for l in labels]

        optimizer.zero_grad()
        predicted_locs, predicted_scores = model(images)
        loss = criterion(predicted_locs, predicted_scores, boxes, labels)
        loss.backward()
        optimizer.step()

        epoch_losses.append(loss.item())

    avg_loss = sum(epoch_losses) / len(epoch_losses)
    print(f"[Эпоха {epoch+1}/{n_epochs}] Средняя потеря на обучении: {avg_loss:.4f}")


ModuleNotFoundError: No module named 'model'

In [ ]:
    # Валидация
    model.eval()
    val_losses = []
    with torch.no_grad():
        for inputs in val_loader:
            images, boxes, labels = inputs
            images = images.to(device)
            boxes = [b.to(device) for b in boxes]
            labels = [l.to(device) for l in labels]

            predicted_locs, predicted_scores = model(images)
            loss = criterion(predicted_locs, predicted_scores, boxes, labels)
            val_losses.append(loss.item())

    avg_val_loss = sum(val_losses) / len(val_losses)
    log.record(epoch + 1, train_loss=avg_loss, valid_loss=avg_val_loss, end='\n')


# Задача 2

Написать код, который отображает 5 случайных изображений из датасета с нарисованными bounding boxes и подписями классов.